<a href="https://colab.research.google.com/github/UnnatiPrema/Binary-Search-1/blob/master/Team28_final_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import Libraries and Mount Google Drive**

The keras and tenserflow libraries were used to create our 1D CNN model.  Sklearn libraries were used to normalize the datasets.  There may be ignorable warnings when importing the libraries.

In [ ]:
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import numpy as np
import scipy.stats as stats

import keras
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, BatchNormalization, Dropout,Conv1D,MaxPooling1D,LSTM,Embedding,GaussianNoise,UpSampling1D

from tensorflow.keras import regularizers

from keras.optimizers import SGD, Adam, RMSprop
import matplotlib.pyplot as plt

from sklearn.utils import class_weight
import math


In [ ]:
from google.colab import files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**Function to Plot Graph**

In [ ]:
# Defining a function for plotting training and validation learning curves
def plot_history(history):
	  # plot loss
    plt.title('Loss')
    plt.plot(history.history['loss'], color='blue', label='train')
    plt.plot(history.history['val_loss'], color='red', label='test')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'])
    plt.show()
    
    # plot accuracy
    plt.title('Accuracy')
    plt.plot(history.history['accuracy'], color='blue', label='train')
    plt.plot(history.history['val_accuracy'], color='red', label='test')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'])
    plt.show()
  

**Functions used to format individual rows into seconds of activity**

This function is used to split the training data into n_step samples.  There is no overlap between samples.

> Indented block



In [ ]:
# split a univariate sequence into samples
def split_sequence(xsequence,ysequence,n_steps):
	X, y = list(), list()
	i = 0
	while i < len(xsequence):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(xsequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x = xsequence[i:end_ix]
		seq_y = np.bincount(ysequence[i:end_ix]).argmax()
		i = i+n_steps
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

This function is used to split the test data into n_step samples.  Again there is no overlap.  Only the X data is returned since this is for a prediction set.

In [ ]:
def split_predictions_sequence(xsequence,n_steps):
	X = list()
	i = 0
	while i < len(xsequence):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(xsequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x = xsequence[i:end_ix]
		i = i+n_steps
		X.append(seq_x)
	return np.array(X)

**Import Data From CSV**

Upload the .csv data to your google drive, adjust the file_path variable if needed. The input values are too large and need to be standardized.

In [ ]:
file_path = '/content/drive/My Drive/ProjectC2Final/final.csv' 

'/content/drive/My Drive/ProjectC2Final/final.csv'

In [ ]:
input = pd.read_csv(file_path,usecols=["xa",	"ya",	"za",	"xg",	"yg",	"zg",	"time",	"subject_name",	"Label"])
input

,xa,ya,za,xg,yg,zg,time,subject_name,Label
0,4.186920,8.344455,2.908057,0.005771,-0.004480,-0.003345,0.02,subject_001__1,0.0
1,4.544637,8.408659,2.890000,0.007967,0.022412,0.001159,0.05,subject_001__1,0.0
2,4.849308,8.411614,2.900692,0.027778,-0.010670,-0.014223,0.07,subject_001__1,0.0
3,4.509190,8.118649,2.847298,0.021577,-0.045498,-0.021111,0.10,subject_001__1,0.0
4,4.226515,8.273807,2.851742,0.012534,0.000445,-0.016830,0.12,subject_001__1,0.0
...,...,...,...,...,...,...,...,...,...
1341612,2.072244,8.908878,-3.500000,0.001351,0.001191,0.001031,1203.32,subject_008__1,0.0
1341613,2.085123,8.915123,-3.520000,0.001918,-0.001147,0.000000,1203.34,subject_008__1,0.0
1341614,2.083774,8.910000,-3.538981,-0.002015,-0.004099,0.001042,1203.37,subject_008__1,0.0
1341615,2.111447,8.908553,-3.535724,0.000183,-0.001673,0.001856,1203.40,subject_008__1,0.0


Encode output with labels and standardize the X data.  

In [ ]:
label = LabelEncoder()
input['Label']=label.fit_transform(input['Label'])
scaler = StandardScaler()
newX=input[['xa','ya','za','xg','yg','zg']]
Y=input['Label'] 
newX = scaler.fit_transform(newX)
scaled_X = pd.DataFrame(data = newX, columns = ['Ax','Ay','Az','Gx','Gy','Gz'])
scaled_X['label'] = Y.values
scaled_X

,Ax,Ay,Az,Gx,Gy,Gz,label
0,0.620543,0.045290,-0.567099,0.000351,-0.018272,0.083457,0
1,0.682938,0.054657,-0.570337,0.001033,-0.001082,0.089309,0
2,0.736081,0.055088,-0.568419,0.007191,-0.022229,0.069323,0
3,0.676755,0.012346,-0.577995,0.005263,-0.044493,0.060374,0
4,0.627449,0.034983,-0.577198,0.002453,-0.015124,0.065937,0
...,...,...,...,...,...,...,...
1341612,0.251684,0.127635,-1.716335,-0.001023,-0.014647,0.089143,0
1341613,0.253931,0.128546,-1.719921,-0.000847,-0.016142,0.087803,0
1341614,0.253695,0.127799,-1.723325,-0.002069,-0.018029,0.089157,0
1341615,0.258522,0.127588,-1.722741,-0.001386,-0.016478,0.090215,0


**Sampling Rate**

Recording rate is the frequency at which the input was recorded, in this example, it was recorded at 40hz, therefore it is set to 40.  Sample_length is the amount of time you want to count as an activity.  The formula is RecordingRate * *Number of seconds*.  So if you wanted a 2 second sample, it would be set to RecordingRate * 2.  

In [ ]:
RecordingRate=40
Sample_length = RecordingRate
n_features = 6

In [ ]:
X = scaled_X[['Ax','Ay','Az','Gx','Gy','Gz']]
Y = scaled_X['label']
Y.value_counts()

0    1006897
3     206436
2      73068
1      55216
Name: label, dtype: int64

In [ ]:
Xarray, Yarray=split_sequence(X.to_numpy(),Y.to_numpy(),Sample_length)

**Cross Validation Test (Optional)**

Test Cross Validation - Dont run this if not needed

In [ ]:
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
cvscores = []

In [ ]:
#Apurva
for train,test in kfold.split(Xarray,Yarray):
  # define model
  model = Sequential()
  model.add(LSTM(units=64, input_shape=[(Xarray[1], Yarray[2])]))
  model.add(Dropout(0.5))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(6,activation='softmax'))
  model.compile(optimizer=Adam(learning_rate=0.00001),loss='categorical_crossentropy',metrics = ['accuracy'])
  model.fit(Xarray[train], Yarray[train], epochs=15, batch_size=10, shuffle = True, verbose=1)
  # evaluate the model
  scores = model.evaluate(Xarray[test], Yarray[test], verbose=0)
  print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
  cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

In [ ]:
for train,test in kfold.split(Xarray,Yarray):
  # define model
  model = Sequential()
  model.add(Conv1D(64,3,activation='relu',input_shape=(Sample_length, n_features),padding='same'))
  model.add(Dropout(0.5))
  model.add(MaxPooling1D(2))

  model.add(Conv1D(128,3,activation='relu',padding='same'))
  model.add(Dropout(0.5))
  model.add(MaxPooling1D(2))

  model.add(Conv1D(128,3,activation='relu',padding='same'))
  model.add(Dropout(0.5))
  model.add(UpSampling1D(2))

  model.add(Conv1D(64,3,activation='relu',padding='same'))
  model.add(Dropout(0.5))
  model.add(UpSampling1D(2))

  model.add(Flatten())
  model.add(Dense(4,activation='softmax'))
  model.compile(optimizer=Adam(learning_rate=0.0001),loss='sparse_categorical_crossentropy',metrics = ['accuracy'])
  model.fit(Xarray[train], Yarray[train], epochs=15, batch_size=10, verbose=1)
  # evaluate the model
  scores = model.evaluate(Xarray[test], Yarray[test], verbose=0)
  print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
  cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))


In [ ]:
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))


**Normal Validation/Training**

Reshape the X and Y Data

In [ ]:
Xarray = Xarray.reshape((Xarray.shape[0]),Xarray.shape[1],n_features)

In [ ]:
Xarray.shape,Yarray.shape

In [ ]:
Counter(Yarray)

**Test and Train Data Split**

Right now the data is split 20% (test_size) variable.

In [ ]:
X_train, X_test,Y_Train,Y_Test = train_test_split(Xarray,Yarray, test_size = 0.2)

In [ ]:
Counter(Y_Train),Counter(Y_Test)

**1D CNN Model**

We tested 2 different 1D CNNs.  The multi-layered CNN performed the best, but took significantly longer to run.

This is the basic 1D CNN, it achieves a 0.62 F1 score with around 20 epochs

In [ ]:
# define model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(Sample_length, n_features),padding='same'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(4,activation='softmax'))
model.compile(optimizer=Adam(learning_rate=0.0001),loss='sparse_categorical_crossentropy',metrics = ['accuracy'])

This is a multi layered 1D CNN, it achieves around a 0.87 F1 score with around 45 epochs.

In [ ]:
model = Sequential()

model.add(Conv1D(256,3,activation='relu',input_shape=(Sample_length, n_features),padding='same'))
model.add(Dropout(0.3))
model.add(MaxPooling1D(2))

model.add(Conv1D(512,3,activation='relu',padding='same'))
model.add(Dropout(0.3))
model.add(MaxPooling1D(2))

model.add(Conv1D(512,3,activation='relu',padding='same'))
model.add(Dropout(0.3))
model.add(UpSampling1D(2))

model.add(Conv1D(256,3,activation='relu',padding='same'))
model.add(Dropout(0.3))
model.add(UpSampling1D(2))

model.add(Flatten())
model.add(Dense(4,activation='softmax'))
model.compile(optimizer=Adam(learning_rate=0.0001),loss='sparse_categorical_crossentropy',metrics = ['accuracy'])

In [ ]:
model.summary()

**Automatic Weights**

This is used to help with the data imbalance.  Use this to automatically calculate what weights to use.  Either use automatic or manual weights, do not use both)

In [ ]:
weights = dict(enumerate(class_weight.compute_class_weight('balanced',np.unique(Y_Train),Y_Train).flatten(),0))
weights

{0: 0.33272159119091316,
 1: 6.043243243243243,
 2: 4.619834710743802,
 3: 1.6325139936724264}

**Train the Model**

In [ ]:
history = model.fit(X_train,Y_Train,epochs=30,validation_data=(X_test,Y_Test),verbose=1,shuffle=True,class_weight=weights)

In [ ]:
plot_history(history)

**Run the predictions for subjects 9 to 12**

Before running the prediction create the samples and standardize the data. Specify the path to the directory containing the prediction datasets.  You do not have to specify each individual csv, only the directory.

In [ ]:
# User must specify this directory
PredictionDirectory = '/content/drive/My Drive/ProjectC2Final/TestData/'
ypredictions = []
for i in range(9,13):
  if i == 9:
    PredictionCSV = PredictionDirectory + '/subject_009_01__x.csv'
    test_input = pd.read_csv(PredictionCSV,names=["Ax","Ay","Az","Gx","Gy","Gz"])
  else:
    PredictionCSV = PredictionDirectory + 'subject_0{0}_01__x.csv'.format(i)
    test_input = pd.read_csv(PredictionCSV,names=["Ax","Ay","Az","Gx","Gy","Gz"])
  newX=test_input[['Ax','Ay','Az','Gx','Gy','Gz']]
  newX = scaler.transform(newX)
  scaled_X = pd.DataFrame(data = newX, columns = ['Ax','Ay','Az','Gx','Gy','Gz'])
  X = scaled_X[['Ax','Ay','Az','Gx','Gy','Gz']]
  X_data = split_predictions_sequence(X.to_numpy(),Sample_length)
  X_data = X_data.reshape((X_data.shape[0]),X_data.shape[1],n_features)
  
  ypredictions.append(model.predict(X_data))

**Transform Y Data**

The Y data is not in the correct form, instead of having 1 output, there are 4 outputs which represent the probability of what class the output represents.  For example Y[0] might look like [0.1,0.2,0.6,0.1].  The third index has the highest probability so the output is set to whatever that index represents.

In [ ]:
class_names = ['0','1','2','3']
argmax_predictions=[]
for j in range (0,4):
  print("Subject {} Predictions Complete".format(j+9))
  #print("---------------------------------------------------")
  argmax_df = pd.DataFrame({'label':[]})
  for i in range (0,len(ypredictions[j])):
    #print(class_names[np.argmax(ypredictions[j][i])])
    add_df = pd.DataFrame({'label':[np.argmax(ypredictions[j][i])]})
    argmax_df = argmax_df.append(add_df)
  argmax_df['label'] = argmax_df['label'].astype('int')
  argmax_predictions.append(argmax_df)



Subject 9 Predictions Complete
Subject 10 Predictions Complete
Subject 11 Predictions Complete
Subject 12 Predictions Complete


**Count the number of outputs for each classification in the prediction set**

This can give you a general idea of whether or not your model is biasing one or two classes

In [ ]:
for j in range(0,4):
  print("Subject {} Labels".format(j+9))
  print(argmax_predictions[j].value_counts())
  print("")

**Create the Prediction CSVs**

Because we divide the inputs into 1 second samples (composed of 40 rows of inputs), there will be less outputs than expected.  The predictions that are created are interpolated to match the expected number of outputs.  This is done by dividing the number of expected outputs by the number of predictions.  For example, if our model produces 100 predictions, but we need to compare those to 1000 outputs, the 100 predicitons would be duplicated by multiples of 10 (1000/100) in a for loop.

In [ ]:
PredictionCSV = pd.read_csv(PredictionDirectory + '/subject_009_01__y_time.csv',names=["label"])
output_length = len(PredictionCSV)
loop_length = math.floor(output_length/len(ypredictions[0]))

In [ ]:
class_names = ['0','1','2','3']

for i in range(9,13):
  print(i)
  if i == 9:
    test_input = pd.read_csv(PredictionDirectory + '/subject_009_01__y_time.csv',names=["label"])
  else:
    test_input = pd.read_csv(PredictionDirectory + '/subject_0{}_01__y_time.csv'.format(i),names=["label"])

  output_length = len(test_input)
  print("Expected Length : ",output_length)
  print("Prediction Length : ",len(ypredictions[i-9]))

  predictions_csv = pd.DataFrame({'Class':[]})
  for x in range(0,len(ypredictions[i-9])):
    new_df = pd.DataFrame({'Class':[class_names[np.argmax(ypredictions[i-9][x])]]})
    for j in range(0,loop_length):
      predictions_csv = predictions_csv.append(new_df)
  print(len(predictions_csv))

  while len(predictions_csv) != output_length:
    new_df = predictions_csv.tail(1)
    predictions_csv = predictions_csv.append(new_df)
  print(len(predictions_csv))
  predictions_csv.to_csv('subject_0{0}_01__y_prediction.csv'.format(i),header=False,index=False)
  files.download('subject_0{0}_01__y_prediction.csv'.format(i))